## Dataset overview
- users.csv: Customer information
- orders.csv: Transaction history
- subscriptions.csv: Subscription data
- preferences.csv: Customer taste preferences
- events.scv: User behavioral events (page views, cart actions)
- voucher_application: Voucher usage data
- user_references.csv: Referral tracking
- products.csv: Product catalog

## Key Question
- Understand what are different groups among current customer base. 
- How do they differ? What are their needs? 
- What are the implications for the marketing strategy in terms of attracting similar new customers and/or retaining current customers?
- How can these insights be used to recommend QLab Tea’s next action for user growth?


In [ ]:
FILTER_WARNINGS = False

In [ ]:
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.DEBUG)

In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


sns.set_style("darkgrid")
sns.set_palette("husl")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

if FILTER_WARNINGS:
    import warnings
    warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

plt.set_loglevel('warning')


logging.info("Setup completed successfully")
logging.debug("Using ENV variables:")
logging.debug(f"FILTER_WARNINGS: {FILTER_WARNINGS}")
logging.warning("This is a warning message.")
logging.error("This is an error message.")
logging.critical("This is a critical message.")

In [ ]:
logging.debug("Loading datasets")

try:
    users_df = pd.read_csv("./original_data/users.csv")
    orders_df = pd.read_csv("./original_data/orders.csv")
    subscriptions_df = pd.read_csv("./original_data/subscriptions.csv")
    products_df = pd.read_csv("./original_data/products.csv")

    preferences_df = pd.read_csv("./original_data/preferences.csv")
    events_df = pd.read_csv("./original_data/events.csv")
    voucher_applications_df = pd.read_csv("./original_data/voucher_applications.csv")
    user_references_df = pd.read_csv("./original_data/user_references.csv")

    logging.info(f"Users: {len(users_df):,} rows")
    logging.info(f"Orders: {len(orders_df):,} rows")
    logging.info(f"Subscriptions: {len(subscriptions_df):,} rows")
    logging.info(f"Products: {len(products_df):,} rows")
    logging.info(f"Preferences: {len(preferences_df):,} rows")
    logging.info(f"Events: {len(events_df):,} rows")
    logging.info(f"Voucher Application: {len(voucher_applications_df):,} rows")
    logging.info(f"User References: {len(user_references_df):,} rows")
except FileNotFoundError as e:
    logging.error(f"Error csv not found: {e}")
    raise
except Exception as e:
    logging.critical(f"Load csv error: {e}")
    raise

In [ ]:
display(users_df.head())
display(orders_df.head())
display(events_df.head())
display(subscriptions_df.head())
logging.debug(users_df.groupby("country").count())
logging.debug(subscriptions_df.groupby("currency").count())
logging.debug(orders_df.groupby("currency").count())

## Data Cleaning

In [ ]:
missing_values = {
    "Users": users_df.isnull().sum(),
    "Orders": orders_df.isnull().sum(),
    "Subscriptions": subscriptions_df.isnull().sum(),
    "Products": products_df.isnull().sum(),
    "Preferences": preferences_df.isnull().sum(),
    "Events": events_df.isnull().sum(),
    "Voucher Application": voucher_applications_df.isnull().sum(),
    "User References": user_references_df.isnull().sum(),
}

for k, v in missing_values.items():
    logging.debug(f"{k}")
    logging.debug(f"{v}\n")

In [ ]:
users_clean_df = users_df.copy()
del users_df

In [ ]:
conversion_rate = {
    "SGD": 1.0,
    "HKD": 0.17,
    "MYR": 0.31,
}


orders_clean_df = orders_df.copy()
# Only keep completed orders
orders_clean_df = orders_clean_df[orders_clean_df["status"].isin(["Shipped", "Delivered"])]
# Remove <=0 order totals
orders_clean_df = orders_clean_df[orders_clean_df["total_incl_tax"]>0]
# Fix dates
orders_clean_df["date_placed"] = pd.to_datetime(orders_clean_df["date_placed"], errors="coerce")
orders_clean_df = orders_clean_df.dropna(subset=["date_placed", "user_id"])

logging.debug(f"Cleaned orders: {len(orders_clean_df):,} rows")
logging.debug(f"Date range: {orders_clean_df['date_placed'].min()} - {orders_clean_df['date_placed'].max()}")
logging.debug(f"Total revenue: {orders_clean_df['total_incl_tax'].sum():.3f}")

def parse_order_items(order_items_json):
    try:
        items = json.loads(order_items_json)
        num_items = sum(int(item.get("quantity", 1)) for item in items)
        product_ids = [item.get("product_id") for item in items]
        return pd.Series({
            "num_items": num_items,
            "product_ids": product_ids
        })
    except:
        return pd.Series({"num_items": 1, "product_ids": []})
    
order_items_parsed = orders_clean_df['order_items'].apply(parse_order_items)
orders_clean_df = pd.concat([orders_clean_df, order_items_parsed], axis=1)

orders_clean_df["total_incl_tax_sgd"] = (
    orders_clean_df["total_incl_tax"]
    * orders_clean_df["currency"].map(conversion_rate)
)

logging.debug("Order items parsing done")
logging.info(f"Average item per order: {orders_clean_df['num_items'].mean():.2f}")
logging.debug(orders_clean_df["order_items"][0])
del orders_df

In [ ]:
subscriptions_clean_df = subscriptions_df.copy()
subscriptions_clean_df = subscriptions_clean_df.dropna(subset = ["user_id"])
date_cols = ["last_order", "next_order", "created", "updated"]
for col in date_cols:
    subscriptions_clean_df[col] = pd.to_datetime(subscriptions_clean_df[col], errors="coerce")
del subscriptions_df

In [ ]:
products_clean_df = products_df.copy()
del products_df

In [ ]:
preferences_clean_df = preferences_df.copy()
preferences_clean_df = preferences_df.dropna(subset=["user_id"])
del preferences_df

In [ ]:
events_clean_df = events_df.copy()
events_clean_df = events_clean_df.dropna(subset = ["user_id", "timestamp"])
events_clean_df["timestamp"] = pd.to_datetime(events_clean_df["timestamp"], errors="coerce")
del events_df

In [ ]:
voucher_applications_clean_df = voucher_applications_df.copy()
voucher_applications_clean_df = voucher_applications_clean_df.dropna(subset=["user_id"])
del voucher_applications_df

In [ ]:
user_references_clean_df = user_references_df.copy()
del user_references_df

In [ ]:
def check_missing_values(df):
    missing_summary = pd.DataFrame({
        'Missing_Count': df.isnull().sum(),
        'Missing_Percentage': (df.isnull().sum() / len(df)) * 100
    }).sort_values('Missing_Percentage', ascending=False)

    logging.info(f"\n{missing_summary[missing_summary['Missing_Count'] > 0]}\n")

# Recheck to see how we did for data cleaning
cleaned_dfs = {
    "Users": users_clean_df,
    "Orders": orders_clean_df,
    "Subscriptions": subscriptions_clean_df,
    "Products": products_clean_df,
    "Preferences": preferences_clean_df,
    "Events": events_clean_df,
    "Voucher Application": voucher_applications_clean_df,
    "User References": user_references_clean_df,
}

for k, v in cleaned_dfs.items():
    logging.info(f"{k}")
    check_missing_values(v)


## Feature Engineering

### 7 types of features
1. RFM (Recency, Frequency, Monetary)
2. Behavioral (purchase patterns)
3. Product Category
4. Subscription
5. Engagement (from events)
6. Promotion (voucher usage)
7. Social (referrals)

#### RFM
- recency_days = (reference_date - max(order_date)) for each customer
- frequency_orders = count of orders per customer
- monetary_total = sum of order totals amount per customer
- avg_order_value = monetary_total/frequency_orders

In [ ]:
newest_date = orders_clean_df["date_placed"].max()
oldest_date = orders_clean_df["date_placed"].min()

logging.debug(f"Reference date: {newest_date}")

# def sigmoid(x):
#     return 1 / (1 + np.exp(-x))

# def sigmoid_series(series, steepness):
#     series_range = series.max() - series.min() or pd.Timedelta(days=1)
#     return sigmoid( 
#         steepness * (
#         (
#             (series - series.min()) / series_range
#         ) 
#         - 0.5)
#     )
#
# orders_clean_df["weighted_total_incl_tax_sgd"] = (
#     orders_clean_df["total_incl_tax_sgd"]
#     * sigmoid_series(orders_clean_df["date_placed"], 12)
# )

def arbitrary_date_weights(dates):
    year = dates.dt.year
    return np.select(
        [year <= 2019, year <= 2020, year <= 2021],
        [0.2, 0.5, 0.8],
        default=1.0,
    )

orders_clean_df["monetary_weight"] = arbitrary_date_weights(orders_clean_df["date_placed"])

orders_clean_df["weighted_total_incl_tax_sgd"] = (
    orders_clean_df["total_incl_tax_sgd"] * orders_clean_df["monetary_weight"]
)

rfm_features = orders_clean_df.groupby("user_id").agg(
    {
        "date_placed": [
            lambda x: (newest_date - x.max()).days,                     # Recency
            lambda x: len(x) / max((x.max() - x.min()).days, 1)         # Frequency
        ],
        "weighted_total_incl_tax_sgd": "sum" # Monetary
    }
).reset_index()

rfm_features.columns = ["user_id", "recency_days", "frequency_orders", "monetary_total"]

order_counts = orders_clean_df.groupby("user_id")["id"].count().rename("order_count")
rfm_features = rfm_features.join(order_counts, on="user_id")

rfm_features["avg_order_value"] = (
    rfm_features["monetary_total"] / rfm_features["order_count"].clip(lower=1)
)

rfm_features["avg_order_value"] = rfm_features["monetary_total"]/rfm_features["frequency_orders"]


logging.debug(f"\n{rfm_features.describe()}")
display(rfm_features.tail())

#### Behavioral
- avg_items_per_order = Average number of items purchased per order
- total_items = Total number of items ever purchased
- num_addresses = Number of different shipping addresses (possible gifting or multi location)
- primary_currency = Most commonly used currency
- order_regularity_std = Standard deviation of days between orders (lower indicate regular purchases)

In [ ]:
behavioral_features = orders_clean_df.groupby("user_id").agg({
    "num_items": ["mean", "sum"], # average number of items per order, total items
    "shipping_postal_code": "nunique", # number of addresses
    "currency": lambda x: x.mode()[0] if len(x.mode()) > 0 else "SGD", # Most common currency type
    }
).reset_index()

behavioral_features.columns = ["user_id",
                               "avg_items_per_order",
                               "total_items",
                               "num_addresses",
                               "primary_currency_behavioral",
                               #"order_regularity_std"
                               ]

def calc_order_regularity(user_orders):
    if len(user_orders) < 2: return 0
    dates = user_orders.sort_values("date_placed")["date_placed"]
    days_between = dates.diff().dt.days.dropna()
    return days_between.std() if len(days_between) > 0 else 0

order_regularity = orders_clean_df.groupby("user_id").apply(calc_order_regularity).reset_index()
order_regularity.columns = ["user_id", "order_regularity_std"]

behavioral_features = behavioral_features.merge(order_regularity, on="user_id")

logging.debug(f"\n{behavioral_features.describe()}")
logging.debug(f"\n{behavioral_features.head(10)}")

#### Product Category
- tea_pod_pct = Percentage of purchases that are tea pods
- tea_bag_pct = Percentage of purchases that are tea bags
- merchandise_pct = Percentage of purchases that are merchandise
- bundle_pct = Percentage of purchases that are bundles/variety packs

In [ ]:
# NOTE: Might need to check through this again to confirm

def categorize_product(title):
    title_lower = str(title).lower()

    match_strings_pod = ("teapods", "pods")
    match_strings_merch = ("bag", "qlab")
    match_strings_subscription = ("subscription",)
    match_strings_bundle = ("bundle", "set", "pack", "variety")

    for keyterm in match_strings_pod:
        if keyterm in title_lower:
            return "tea_pod"
    
    for keyterm in match_strings_merch:
        if keyterm in title_lower:
            return "merchandise"
        
    for keyterm in match_strings_subscription:
        if keyterm in title_lower:
            return "subscription"
    
    for keyterm in match_strings_bundle:
        if keyterm in title_lower:
            return "bundle"
        
    return "tea_bag"

products_clean_df["category"] = products_clean_df["title"].apply(categorize_product)
logging.debug(products_clean_df["category"].value_counts())

def get_product_categories(product_ids_list):
    if not isinstance(product_ids_list, list) or len(product_ids_list)==0: return []

    categories = []
    for pid in product_ids_list:
        if pid is None:
            continue

        try:
            pid_int = int(pid)
            matching_products = products_clean_df[products_clean_df["product_id"]==pid_int]
            if len(matching_products) > 0:
                categories.append(matching_products.iloc[0]["category"])
        except (ValueError, TypeError):
            continue
    return categories


product_features_list = []
total_customers = len(rfm_features)

for idx, user_id in enumerate(rfm_features["user_id"]):
    if (idx+1) %1000 == 0:
        logging.debug(f"Processing {idx+1}/{total_customers} customers")
    user_orders = orders_clean_df[orders_clean_df["user_id"] == user_id]

    all_product_ids = []
    for pids in user_orders["product_ids"]:
        if isinstance(pids, list):
            all_product_ids.extend(pids)

    categories = get_product_categories(all_product_ids)

    total = len(categories) if len(categories) > 0 else 1

    product_features_list.append({
        "user_id": user_id,
        "tea_pod_pct": (categories.count("tea_pod") / total) * 100,
        "tea_bag_pct": (categories.count("tea_bag") / total) * 100,
        "merchandise_pct": (categories.count("merchandise") / total) * 100,
        "bundle_pct": (categories.count("bundle") / total) * 100
        }
    )

product_features = pd.DataFrame(product_features_list)

logging.debug(product_features.describe())
display(product_features.head(10))


#### Subscription

In [ ]:
subscription_features = subscriptions_clean_df.copy()

subscription_features["is_active"] = subscription_features["status"].str.lower().eq("active").astype(int)
subscription_features["surprise_flag"] = subscription_features["surprise"].str.strip().str.lower().eq("yes").astype(int)
subscription_features["order_gap_days"] = (subscription_features["next_order"] - subscription_features["last_order"]).dt.days
subscription_features["subscription_age_days"] = (pd.Timestamp("today") - subscription_features["created"]).dt.days

# --- aggregate per user ---
subscription_features = subscription_features.groupby("user_id").agg(
    total_subscription_records=("product_id", "count"),
    unique_products=("product_id", "nunique"),
    num_shipping_addresses=("shipping_postal_code", "nunique"),
    num_countries=("country", "nunique"),
    avg_interval=("interval", "mean"),
    interval_std=("interval", "std"),
    active_subscriptions=("is_active", "sum"),
    active_ratio=("is_active", "mean"),
    primary_shipping_method=("shipping_method", lambda x: x.mode()[0] if len(x.mode()) else "unknown"),
    primary_currency_subscription=("currency", lambda x: x.mode()[0] if len(x.mode()) else "SGD"),
    surprise_ratio=("surprise_flag", "mean"),
    avg_order_gap_days=("order_gap_days", "mean"),
    order_gap_std=("order_gap_days", "std"),
    avg_subscription_age_days=("subscription_age_days", "mean"),
    first_subscription_created=("created", "min"),
    latest_subscription_update=("updated", "max"),
).reset_index()

logging.debug(f"\n{subscription_features.describe()}")
logging.debug(f"\n{subscription_features.head(10)}")

#### Engagement

In [ ]:
logging.debug(events_clean_df.head(10))
logging.debug(events_clean_df["event"].unique())

events_df = events_clean_df.copy()
events_df["event_date"] = events_df["timestamp"].dt.date

high_intent_events = {
    "ACTION_VIEW_PRODUCT",
    "ACTION_ADD_ITEM_TO_CART",
    "ACTION_VOUCHER_APPLY",
    "PURCHASED_GIFT_CARD",
}

events_df["is_high_intent"] = events_df["event"].isin(high_intent_events).astype(int)

last_ts = events_df["timestamp"].max().normalize()

engagement_core = events_df.groupby("user_id").agg(
    total_events=("event", "count"),
    unique_event_types=("event", "nunique"),
    active_days=("event_date", "nunique"),
    high_intent_events=("is_high_intent", "sum"),
    last_event_ts=("timestamp", "max"),
).reset_index()

total_sessions = (events_df[events_df["event"]=="ACTION_LOGIN"].groupby("user_id").size().reset_index(name="total_sessions"))

engagement_core["events_per_active_day"] = (
    engagement_core["total_events"] / engagement_core["active_days"].clip(lower=1)
)
engagement_core["high_intent_ratio"] = (
    engagement_core["high_intent_events"] / engagement_core["total_events"].clip(lower=1)
)
engagement_core["days_since_last_event"] = (
    last_ts - engagement_core["last_event_ts"]
).dt.days

key_event_counts = (
    events_df[events_df["event"].isin(high_intent_events)]
    .pivot_table(index="user_id", columns="event", values="timestamp", aggfunc="count", fill_value=0)
    .add_prefix("event_")
    .reset_index()
)

engagement_features = (
    engagement_core
    .merge(total_sessions, on="user_id", how="left")
    .merge(key_event_counts, on="user_id", how="left")
    .fillna(0)
)

logging.debug(f"\n{engagement_features.describe(include='all')}")
logging.debug(f"\n{engagement_features.head(10)}")

#### Promotion (Voucher Usage)

In [ ]:
import pandas as pd
import numpy as np

logging.info("--- Starting Voucher Feature Engineering (Group Part) ---")

try:
    vouchers_df = pd.read_csv("./original_data/vouchers.csv")
    # 提取所需字段，不修改原始文件
    # 假设 'total_discount' 是固定折扣金额，'date_created' 用于计算时效性
    vouchers_meta = vouchers_df[['id', 'total_discount', 'date_created']].rename(
        columns={'id': 'voucher_id', 'total_discount': 'discount_val'}
    )
    vouchers_meta['discount_val'] = vouchers_meta['discount_val'].fillna(0)
    vouchers_meta['date_created'] = pd.to_datetime(vouchers_meta['date_created'])
except FileNotFoundError:
    logging.error("vouchers.csv not found!")
    raise

# 2. 基础关联：Orders + Applications + Vouchers
# 严守不改名原则：Orders(id) <-> Applications(order_id)
promo_merged = orders_clean_df[['id', 'user_id', 'total_incl_tax']].merge(
    voucher_applications_clean_df[['order_id', 'voucher_id', 'created']], 
    left_on='id', 
    right_on='order_id', 
    how='left'
)

# 关联 Voucher 详情
promo_merged = promo_merged.merge(vouchers_meta, on='voucher_id', how='left')

# 3. 计算订单级指标
promo_merged['has_voucher'] = promo_merged['voucher_id'].notna().astype(int)
promo_merged['savings'] = np.where(promo_merged['has_voucher'] == 1, promo_merged['discount_val'], 0)

# 计算 lag_days (领券/用券 滞后天数): 使用时间(created) - 发布时间(date_created)
promo_merged['created'] = pd.to_datetime(promo_merged['created'])
promo_merged['lag_days'] = (promo_merged['created'] - promo_merged['date_created']).dt.days

# 4. 用户级聚合 (User Aggregation)
voucher_stats = promo_merged.groupby('user_id').agg(
    total_orders=('id', 'count'),
    voucher_orders_count=('has_voucher', 'sum'),
    total_savings=('savings', 'sum'),
    total_revenue=('total_incl_tax', 'sum'),
    unique_vouchers_used=('voucher_id', 'nunique'), # 进阶：用过几种不同的券
    avg_voucher_lag_days=('lag_days', 'mean')       # 进阶：平均响应天数
).reset_index()

# 5. 计算比率特征
voucher_stats['voucher_usage_rate'] = voucher_stats['voucher_orders_count'] / voucher_stats['total_orders']
voucher_stats['discount_savings_ratio'] = voucher_stats.apply(
    lambda x: x['total_savings'] / x['total_revenue'] if x['total_revenue'] > 0 else 0, axis=1
)

# 6. 缺失值处理
voucher_stats['avg_voucher_lag_days'] = voucher_stats['avg_voucher_lag_days'].fillna(-1)
voucher_stats[['voucher_orders_count', 'total_savings', 'unique_vouchers_used']] = \
    voucher_stats[['voucher_orders_count', 'total_savings', 'unique_vouchers_used']].fillna(0)

# 7. 输出最终表 (只保留特征列 + user_id)
voucher_features = voucher_stats[[
    'user_id', 
    'voucher_orders_count', 
    'voucher_usage_rate', 
    'total_savings', 
    'discount_savings_ratio',
    'unique_vouchers_used',
    'avg_voucher_lag_days'
]]

logging.info(f"Voucher Part Complete. Shape: {voucher_features.shape}")
display(voucher_features.head())
# 1. 统计有多少人真正用过券
used_count = len(voucher_features[voucher_features['voucher_orders_count'] > 0])
print(f"用过优惠券的用户数量: {used_count}")
print(f"占比: {used_count / len(voucher_features):.2%}")

# 2. 展示前 5 个用过券的“真实数据”
print("\n--- 用过券的用户示例 (非0数据) ---")
display(voucher_features[voucher_features['voucher_orders_count'] > 0].head())

#### Social (Referrals)

In [ ]:
import pandas as pd
import numpy as np

logging.info("--- Starting Social Feature Engineering (Group Part) ---")

# 1. 推荐人维度 (Referrer): 统计拉了多少人
# 统计 referred_by 出现的次数
referrer_stats = user_references_clean_df.groupby('referred_by').size().reset_index(name='num_referred_users')

# 2. 被推荐人维度 (Referee): 谁是被拉进来的
referee_users_set = set(user_references_clean_df['user_id'].unique())

# 3. 进阶时间特征: 注册后多久开始拉人 (Days to First Referral)
# 找到每个推荐人的第一次推荐时间
first_ref_time = user_references_clean_df.groupby('referred_by')['created'].min().reset_index(name='first_referral_date')
first_ref_time['first_referral_date'] = pd.to_datetime(first_ref_time['first_referral_date'])

# 4. 合并回用户表 (User Base)
# 以 Users 表为基准，因为我们需要 date_signed_up 来计算时间差
# 严守不改名原则：Users(id) <-> user_references(referred_by)
social_base = users_clean_df[['id', 'date_signed_up']].drop_duplicates()

# 合并推荐数量
social_base = social_base.merge(
    referrer_stats,
    left_on='id',
    right_on='referred_by',
    how='left'
)

# 合并第一次推荐时间
social_base = social_base.merge(
    first_ref_time,
    left_on='id',
    right_on='referred_by',
    how='left'
)

# 5. 计算特征
# Feature A: 成功推荐人数 (空值填0)
social_base['num_referred_users'] = social_base['num_referred_users'].fillna(0).astype(int)

# Feature B: 是否被推荐用户 (0/1)
social_base['is_referred_user'] = social_base['id'].apply(lambda x: 1 if x in referee_users_set else 0)

# Feature C: 注册到首次推荐的天数差 (Days to First Referral)
social_base['date_signed_up'] = pd.to_datetime(social_base['date_signed_up'])
social_base['days_to_first_referral'] = (social_base['first_referral_date'] - social_base['date_signed_up']).dt.days

# 处理时间差的空值 (没推荐过人 = -1)
social_base['days_to_first_referral'] = social_base['days_to_first_referral'].fillna(-1)

# 6. 清理列 (删除合并产生的辅助列)
cols_to_drop = [c for c in social_base.columns if 'referred_by' in c or 'date_' in c]
social_features_clean = social_base.drop(columns=cols_to_drop)

# 重命名 id 为 user_id 以便最后统一合并 (可选，如果组长要求保留 id 则不跑这行)
social_features = social_features_clean.rename(columns={'id': 'user_id'})

logging.info(f"Social Part Complete. Shape: {social_features.shape}")
display(social_features.head())

# 1. 看看有多少“带货王” (成功推荐过别人)
koc_df = social_features[social_features['num_referred_users'] > 0]
print(f"成功推荐过他人的用户数: {len(koc_df)}")

# 2. 看看有多少“被安利的人” (通过推荐注册)
referee_df = social_features[social_features['is_referred_user'] == 1]
print(f"通过推荐注册的用户数: {len(referee_df)}")

# 3. 展示一下真实的社交数据
print("\n--- 社交达人 (KOC) 示例 ---")
display(koc_df.head())

print("\n--- 刚注册就拉人的用户 (时间差小) ---")
# 筛选出推荐过人，且时间差不为 -1 的
active_referrers = social_features[social_features['days_to_first_referral'] >= 0]
display(active_referrers.sort_values('days_to_first_referral').head())

#### SKIPPED FEATURES
- NOTE: 一个taste的feature可能有用，McKinsey的好像是有一个taste_feature

### Combine all features

In [ ]:
customer_features = rfm_features.copy()
customer_features = customer_features.merge(behavioral_features, on="user_id", how="left")
customer_features = customer_features.merge(product_features, on="user_id", how="left")
customer_features = customer_features.merge(subscription_features, on="user_id", how="left")
customer_features = customer_features.merge(engagement_features, on="user_id", how="left")
customer_features = customer_features.merge(voucher_features, on="user_id", how="left")
customer_features = customer_features.merge(social_features, on="user_id", how="left")
#customer_features = customer_features.merge(taste_features, on="user_id", how="left")


customer_features["conversion_rate"] = customer_features.apply(
    lambda row: row["frequency_orders"] / row["total_sessions"] if row["total_sessions"] > 0 else 0,
    axis = 1,
)

fill_zero_cols = [

]

for col in fill_zero_cols:
    if col in customer_features.columns:
        customer_features[col] = customer_features[col].fillna(0)

max_regularity = customer_features["order_regularity_std"].max()

if pd.isna(max_regularity):
    max_regularity = 999

customer_features["order_regularity_std"] = customer_features["order_regularity_std"].fillna(max_regularity)

logging.debug(customer_features.columns.tolist())
logging.debug(customer_features.head())
logging.debug(customer_features.describe())

### Post combination processing

In [ ]:
# Since there are values in primary_currency that are NaN, we use the other primary_currency column to fill

behavioral = customer_features["primary_currency_behavioral"]
subscription = customer_features["primary_currency_subscription"]

customer_features["primary_currency_behavioral"] = behavioral.fillna(subscription)
customer_features["primary_currency_subscription"] = subscription.fillna(
    customer_features["primary_currency_behavioral"]
)

## Exploratory Data Analysis

### Split numerical and categorical features

In [ ]:
# Separate numerical and categorical features
num_features = customer_features.select_dtypes(include=[np.number]).columns.tolist()
cat_features = customer_features.select_dtypes(include=['object', 'category']).columns.tolist()

logging.info(f"Numerical features ({len(num_features)}): {num_features}")
logging.debug(f"Categorical features ({len(cat_features)}): {cat_features}")

# Detailed summary for numerical features
logging.debug(customer_features[num_features].describe())

# Check for unique values in categorical features
for col in cat_features:
    logging.debug(f"{col}: {customer_features[col].nunique()} unique values")



# Special consideration for primary currency mismatch
# 1) Boolean flag per row
customer_features["currency_mismatch"] = (
    customer_features["primary_currency_behavioral"]
    != customer_features["primary_currency_subscription"]
)

# 2) Inspect any mismatches
mismatched = customer_features[customer_features["currency_mismatch"]]

print(f"Mismatched customers: {len(mismatched)}")
print(mismatched[["user_id", "primary_currency_behavioral", "primary_currency_subscription"]].head())
print(customer_features.groupby("primary_currency_behavioral").count()["user_id"])

### Check missing values

In [ ]:
# Calculate missing values percentage
missing_summary = pd.DataFrame({
    'Missing_Count': customer_features.isnull().sum(),
    'Missing_Percentage': (customer_features.isnull().sum() / len(customer_features)) * 100
}).sort_values('Missing_Percentage', ascending=False)

print("Missing Values Summary:")
print(missing_summary[missing_summary['Missing_Count'] > 0])

# Visualize missing values pattern
plt.figure(figsize=(12, 6))
sns.heatmap(customer_features.isnull(), cbar=True, yticklabels=False, cmap='viridis')
plt.title('Missing Values Pattern')
plt.tight_layout()
plt.show()

### Outlier Detection

In [ ]:
outlier_summary = {}
for col in num_features:
    Q1 = customer_features[col].quantile(0.25)
    Q3 = customer_features[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = customer_features[(customer_features[col] < lower_bound) | (customer_features[col] > upper_bound)][col]
    outlier_percentage = (len(outliers) / len(customer_features)) * 100
    outlier_summary[col] = outlier_percentage

outlier_df = pd.DataFrame.from_dict(outlier_summary, orient='index', 
                                   columns=['Outlier_Percentage'])
print("Outlier Summary (IQR method):")
print(outlier_df.sort_values('Outlier_Percentage', ascending=False))

### Distribution of key customer features

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10, 15))
fig.suptitle("Distribution of Key Customer Features", fontsize=15, fontweight="bold")

# Recency
axes[0, 1].hist(customer_features["recency_days"], bins=50, edgecolor="black")
axes[0, 1].set_title("Recency (Days since last order)")
axes[0, 1].set_xlabel("Days")
axes[0, 1].set_ylabel("Frequency")

# Frequency
axes[0, 1].hist(customer_features["frequency_orders"], bins=50, edgecolor="black")
axes[0, 1].set_title("Frequency (Numbers of Orders)")
axes[0, 1].set_xlabel("Number of Orders")
axes[0, 1].set_ylabel("Frequency")

# Monetary
axes[0, 2].hist(customer_features["monetary_total"], bins=50, edgecolor="black")
axes[0, 2].set_title("Monetary (Total Spend)")
axes[0, 2].set_xlabel("Number Spend ($)")
axes[0, 2].set_ylabel("Frequency")

# Subscription status
# subscription_counts = customer_features["has_subscription"].value_counts()
customer_features["has_subscription"] = (customer_features["active_subscriptions"].fillna(0) > 0).astype(int)
subscription_counts = customer_features["has_subscription"].value_counts()
axes[1, 0].bar(["No Subscription", "Has Subscription"], subscription_counts.values)
axes[1, 0].set_title("Subscription Status Distribution")
axes[1, 0].set_ylabel("Number of Customers")

# Engagement
axes[1, 1].hist(customer_features["total_events"], bins=50, edgecolor="black")
axes[1, 1].set_title("Total Engagement Events")
axes[1, 1].set_xlabel("Number of Events")
axes[1, 1].set_ylabel("Frequency")

# Voucher usage
#voucher_counts = (customer_features["is_voucher_user"] > 0).value_counts()
voucher_counts = (customer_features["voucher_orders_count"].fillna(0) > 0).value_counts()
axes[1, 2].bar(["No Vouchers", "Used Vouchers"], voucher_counts.values)
axes[1, 2].set_title("Voucher Usage Distribution")
axes[1, 2].set_ylabel("Number of Customers")

# Taste Preferences,   这个地方我改了，注意  注意  注意
customer_features['has_preferences'] = customer_features['user_id'].isin(preferences_clean_df['user_id']).astype(int)
taste_counts = (customer_features["has_preferences"] >0).value_counts()
axes[2, 2].bar(["No Preferences", "Has Preferences"], taste_counts.values)
axes[2, 2].set_title("Taste Preferences Set")
axes[2, 2].set_ylabel("Number of Customers")

plt.tight_layout()
plt.show()

### Correlation Matrix

In [ ]:
plt.figure(figsize=(16, 14))

#这里新加了selct_dtypes去取数值类型，因为只能计算数值类型的相关性

numeric_features = customer_features.select_dtypes(include=[np.number]).drop("user_id", axis=1, errors="ignore")

correlation_matrix = numeric_features.corr()
sns.heatmap(correlation_matrix, annot=False, fmt=".2f", cmap="coolwarm", center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.0})
plt.title("Feature Correlation Matrix", fontsize=14, fontweight="bold", pad=20)
plt.tight_layout()
plt.show()

## K-Means Clustering

In [ ]:
clustering_features = customer_features.drop(["user_id", "primary_currency"], axis=1, errors="ignore")

logging.debug(f"Features for clustering: {len(clustering_features.columns)} features")
logging.debug(f"Shape: {clustering_features.shape}")

missing_count = clustering_features.isnull().sum().sum()
logging.debug(f"Missing values: {missing_count}")

if missing_count > 0:
    logging.warning("Found missing values")
    # 这里我也改了，非数据和数据类型不同的处理方式  注意 注意  注意
    # 只对数值型列填充中位数
    num_cols = clustering_features.select_dtypes(include=[np.number]).columns
    cluster_features = clustering_features.copy()
    cluster_features[num_cols] = cluster_features[num_cols].fillna(cluster_features[num_cols].median())
    # 对非数值型列用众数填充
    non_num_cols = clustering_features.select_dtypes(exclude=[np.number]).columns
    for col in non_num_cols:
        mode_val = clustering_features[col].mode()
        if not mode_val.empty:
            cluster_features[col] = cluster_features[col].fillna(mode_val[0])
        else:
            cluster_features[col] = cluster_features[col].fillna("unknown")
    logging.warning("Missing values filled with median/mode")
else:
    cluster_features = clustering_features.copy()

In [ ]:

#注意这里只能是数据类型  所以又改了  注意  注意  注意
numeric_cluster_features = cluster_features.select_dtypes(include=[np.number])
scaler = StandardScaler()
features_scaled = scaler.fit_transform(numeric_cluster_features)

logging.debug("Clutering Features scaled using StandardScaler")
logging.debug(f"Scaled features shape: {features_scaled.shape}")
logging.debug(f"Scaled features mean: {features_scaled.mean():.4f}") # Should be near 0
logging.debug(f"Scaled features std: {features_scaled.std():.4f}") # Should be near 1

### Testing Different Approaches

In [ ]:
random_state = 1
inertias = []
silhouette_scores_list = []
k_range = range(2, 11)

for k in k_range:
    logging.debug(f"Testing k={k}")

    k_means = KMeans(n_clusters=k, random_state=random_state, n_init=10)
    k_means.fit(features_scaled)

    inertias.append(k_means.inertia_)
    silhouette_scores_list.append(silhouette_score(features_scaled, k_means.labels_))

    logging.info(f"K={k} | Inertia={inertias[-1]:.3f} | Silhouette: {silhouette_scores_list[-1]:.3f}")


plt, axes = plt.subplots(1, 2, figsize=(14, 9))

axes[1].plot(k_range, inertias, "ro-", linewidth=2, markersize=3)
axes[1].set_xlabel("Number of Clusters (k)", fontsize=12)
axes[1].set_ylabel("Inertia (Within-cluster sum of squares)", fontsize=12)
axes[0].set_title("Elbow method", fontsize=14, fontweight="bold")
axes[0].grid(True, alpha=0.3)

axes[1].plot(k_range, silhouette_scores_list, "ro-", linewidth=2, markersize=3)
axes[1].set_xlabel("Number of Clusters (k)", fontsize=12)
axes[1].set_ylabel("Silhouette Score", fontsize=12)
axes[1].set_title("Silhouette Score Method", fontsize=14, fontweight="bold")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
optimal_k = 4

logging.debug(f"Fitting K-Means with k={optimal_k}")

k_means_final = KMeans(n_clusters=optimal_k, random_state=random_state, n_init=10)
cluster_labels = k_means_final.fit_predict(features_scaled)

customer_features["cluster"] = cluster_labels

logging.info(f"Final inertia: {k_means_final.inertia_:.2f}")
logging.info(f"Silhouette Score: {silhouette_score(features_scaled, cluster_labels):.3f}")

cluster_counts = customer_features["cluster"].value_counts().sort_index()
for cluster_id, count in cluster_counts.items():
    percentage = (count / len(customer_features)) * 100
    logging.info(f"Cluster {cluster_id}: {count} customers ({percentage:.1f}%)")

## Cluster Analysis and Profiling
- 视频有点难看，代码很多，我懒得再抄了
- 而且我们模型不会一样
- 可以自己研究一下

In [ ]:
# 要先做好一个cluster_profiles
# 让后这个是McKinsey他们的定义，只是参考

def interpret_cluster(cluster_id, profile):
    row = profile.loc[cluster_id]

    high_freq = row["frequency_orders"] > cluster_profiles["frequency_orders"].median()
    high_monetary = row["monetary_total"] > cluster_profiles["monetary_total"].median()
    low_recency = row["recency_days"] < cluster_profile["recency_days"].median()
    has_sub = row["has_subsciption"] > 0.5
    high_engagement = row["total_events"] > cluster_profiles["total_events"].median()

    if high_freq and high_monetary and has_sub:
        name = "VIP Champions"
        description = "High value loyal customers with active subscriptions"
    elif high_freq and low_recency:
        name = "Loyal Regulars"
        description = "Frequent buyers who purchase regularly"
    elif not low_recency and not high_freq:
        name = "At-Risk / Hibernating"
        description = "Previously active but haven't purchased recently"
    else:
        name = "Casual Explorers"
        description = "Infrequent buyers, potential growth"
    return name, description

for cluster_id in range(optimal_k):
    name, description = interpret_cluster(cluster_id, cluster_profiles)
    cluster_names[cluster_id] = name
    cluster_descriptions[cluster_id] = description

    size = (customer_features["cluster"] == cluster_id).sum()
    percentage = (size / len(customer_features)) * 100

    revenue_info = revenue_analysis(revenue_analysis["cluster"] == cluster_id).iloc[0]
    logging.info(f"Cluster {cluster_id}: {name}")
    logging.info(f"Description: {description}")
    logging.info(f"Size: {size} customers ({percentage:.1f}%)")
    # logging.info(f"Revenue contribution: {revenue_info["total_revenue"]:.2f} ({revenue_info["total_pct"]:.2f}% of total)")


    row = cluster_profiles.loc[cluster_id]
    # log out avg recency, avg frequency, avg total spend, avg order value
    # subscription rate, avg engagement events, voucher usage rate
    # referral rate



## Business Recommendations

In [ ]:
recommendations = []

for _, rev_row in revenue_analysis.iterrows():
    cluster_id = int(rev_row["cluster"])
    row = cluster_profiles.loc[cluster_id]
    name = cluster_names[cluster_id]
    descriptions = cluster_descriptions[cluster_id]
    size = int(rev_row["customer_count"])

    strategies = []

    if row["monetary_total"] > cluster_profiles["monetary_total"].quantile(0.75):
        strategies.append("VIP program: Exclusive early access to new products")
        strategies.append("Premium Offerings: Introduce limited edition or premium...")
    # something like this...


# Also find the priority of each cluster and show why(size%, revenue contribution, avg revenue per customer)
# sort by this priority